# Data Overview & Schema Exploration

## Database Connection

In [1]:
import sqlite3
import pandas as pd

DB_PATH = "D:\code\py\daproject\data\olist.sqlite"
conn = sqlite3.connect(DB_PATH)


## List of Tables

In [3]:
tables = pd.read_sql("""
SELECT name
FROM sqlite_master
WHERE type = 'table'
ORDER BY name;
""", conn)

tables


,name
0,customers
1,geolocation
2,leads_closed
3,leads_qualified
4,order_items
5,order_payments
6,order_reviews
7,orders
8,product_category_name_translation
9,products


## Table Schemas
- orders
- order_items
- customers
- products


In [4]:
def show_schema(table_name):
    return pd.read_sql(
        f"PRAGMA table_info({table_name});",
        conn
    )

show_schema("orders")
show_schema("order_items")
show_schema("customers")
show_schema("products")


,cid,name,type,notnull,dflt_value,pk
0,0,product_id,TEXT,0,None,0
1,1,product_category_name,TEXT,0,None,0
2,2,product_name_lenght,REAL,0,None,0
3,3,product_description_lenght,REAL,0,None,0
4,4,product_photos_qty,REAL,0,None,0
5,5,product_weight_g,REAL,0,None,0
6,6,product_length_cm,REAL,0,None,0
7,7,product_height_cm,REAL,0,None,0
8,8,product_width_cm,REAL,0,None,0


## Table Row Counts

In [5]:
def table_row_counts(table_list):
    rows = []
    for table in table_list:
        cnt = pd.read_sql(
            f"SELECT COUNT(*) AS row_count FROM {table};",
            conn
        )["row_count"][0]
        rows.append({"table_name": table, "row_count": cnt})
    return pd.DataFrame(rows)

table_row_counts(tables["name"].tolist())


,table_name,row_count
0,customers,99441
1,geolocation,1000163
2,leads_closed,842
3,leads_qualified,8000
4,order_items,112650
5,order_payments,103886
6,order_reviews,99224
7,orders,99441
8,product_category_name_translation,71
9,products,32951


## Table Relationships Summary

In [6]:
pd.read_sql("""
SELECT COUNT(*) AS cnt
FROM orders o
JOIN order_items oi
  ON o.order_id = oi.order_id;
""", conn)


,cnt
0,112650


## Data Quality Check

Before performing data cleaning and analysis, basic data quality checks
are conducted to identify missing values, duplicates, and obvious anomalies.
At this stage, issues are identified but not corrected.


### Missing Values

Check whether key business fields contain NULL values.
Missing values in identifiers or timestamps may affect joins and metric calculations.

In [9]:
pd.read_sql("""
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS missing_customer_id,
    SUM(CASE WHEN order_purchase_timestamp IS NULL THEN 1 ELSE 0 END) AS missing_purchase_time
FROM orders;
""", conn)


,total_rows,missing_customer_id,missing_purchase_time
0,99441,0,0


### Duplicate Records

Verify that primary keys are unique.
For the orders table, each `order_id` should appear only once.

In [10]:
pd.read_sql("""
SELECT order_id, COUNT(*) AS cnt
FROM orders
GROUP BY order_id
HAVING COUNT(*) > 1;
""", conn)


,order_id,cnt


### Invalid Values

Check for obviously invalid numeric values, such as non-positive prices
or negative freight costs.

In [11]:
pd.read_sql("""
SELECT COUNT(*) AS invalid_price_cnt
FROM order_items
WHERE price <= 0;
""", conn)
pd.read_sql("""
SELECT COUNT(*) AS invalid_freight_cnt
FROM order_items
WHERE freight_value < 0;
""", conn)

,invalid_freight_cnt
0,0


### Timestamp Consistency

Verify the logical order of timestamps, such as ensuring delivery dates
do not occur before purchase dates.

In [12]:
pd.read_sql("""
SELECT COUNT(*) AS invalid_time_orders
FROM orders
WHERE order_delivered_customer_date < order_purchase_timestamp;
""", conn)


,invalid_time_orders
0,0


In [1]:
from src.data_scope import get_connection, load_orders_clean

conn = get_connection()
orders = load_orders_clean(conn)

orders["order_status"].unique()


ModuleNotFoundError: No module named 'src'